## Build Functions for ELT

In [7]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [8]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Download, Transform, and Modeling All in One

In [9]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [17]:
f.predictions('nvda')

NVDA 5m Interval Timestamp: 2024-11-11 12:24:40 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.201145,0.165283,0.202703,0.108316
prob_up,0.533684,0.494902,0.416755,0.625
prob_static,0.248876,0.226291,0.21964,0.125
prob_down,0.217441,0.278807,0.363605,0.25
precision,0.429389,0.403774,0.430502,0.363083
recall,0.440313,0.418787,0.436399,0.350294
f1,0.434783,0.411143,0.433431,0.356574
support,"[511.0, 511.0, 510.0]","[511.0, 511.0, 510.0]","[511.0, 511.0, 510.0]","[511.0, 511.0, 510.0]"


NVDA 15m Interval Timestamp: 2024-11-11 12:24:50 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,up,static
kelly_1:2.5,0.106533,0.091892,0.161353,0.194521
prob_up,0.422877,0.356393,0.374757,0.25
prob_static,0.318891,0.282275,0.300834,0.375
prob_down,0.258232,0.361332,0.324409,0.375
precision,0.361809,0.351351,0.400966,0.424658
recall,0.423529,0.389222,0.488235,0.550296
f1,0.390244,0.369318,0.440318,0.479381
support,"[170.0, 169.0, 167.0]","[170.0, 169.0, 167.0]","[170.0, 169.0, 167.0]","[170.0, 169.0, 167.0]"


NVDA 1h Interval Timestamp: 2024-11-11 12:24:53 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,down,down
kelly_1:2.5,0.069018,0.113814,0.117857,0.118689
prob_up,0.364456,0.331875,0.341475,0.375
prob_static,0.329562,0.352577,0.315602,0.125
prob_down,0.305981,0.315547,0.342923,0.5
precision,0.335013,0.36701,0.369898,0.370492
recall,0.309302,0.412037,0.337209,0.262791
f1,0.321644,0.388222,0.352798,0.307483
support,"[430.0, 432.0, 430.0]","[430.0, 432.0, 430.0]","[430.0, 432.0, 430.0]","[430.0, 432.0, 430.0]"


NVDA 1wk Interval Timestamp: 2024-11-11 12:25:12 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,down,static
kelly_1:2.5,-0.005128,0.013636,0.030769,0.078947
prob_up,0.162132,0.294394,0.246998,0.25
prob_static,0.452747,0.282186,0.341324,0.5
prob_down,0.385121,0.42342,0.411678,0.25
precision,0.282051,0.295455,0.307692,0.342105
recall,0.25,0.295455,0.272727,0.443182
f1,0.26506,0.295455,0.289157,0.386139
support,"[87.0, 88.0, 88.0]","[87.0, 88.0, 88.0]","[87.0, 88.0, 88.0]","[87.0, 88.0, 88.0]"


NVDA 1mo Interval Timestamp: 2024-11-11 12:25:15 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.2,0.125,0.2,0.109091
prob_up,0.197726,0.251242,0.263976,0.125
prob_static,0.653981,0.533167,0.452389,0.5
prob_down,0.148293,0.21559,0.283635,0.375
precision,0.428571,0.375,0.428571,0.363636
recall,0.6,0.45,0.6,0.6
f1,0.5,0.409091,0.5,0.45283
support,"[18.0, 20.0, 18.0]","[18.0, 20.0, 18.0]","[18.0, 20.0, 18.0]","[18.0, 20.0, 18.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')